### <center> Assignment 2: word2vec </center>

In [223]:
import pandas as pd
import numpy as np
import re

In [224]:
from nltk.corpus import stopwords

#in this function we replace some symbols like "(, ), [, ], {, }, |, \, /, ','" etc by space
#DO NOT REMOVE STOPWORDS FROM TEXT

REPLACE_BY_SPACE_RE = re.compile(r'[/(){}\[\]\|@,;\.:\->_\?\'\"\*\d+]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def prepare_txt(text):
    text = REPLACE_BY_SPACE_RE.sub(" ", text)
    text = re.sub("  ", " ", text)
    return text

In [225]:
def create_raw_train_data(file):
    with open(file, 'r') as f:
        text = f.read()
    text = prepare_txt(text).lower()
    return text
text = create_raw_train_data("news_dataset1.txt")

#### <center> word2vec summary  </center>
1. Set up a fake problem of determining the context words from a target word *w*.    
2. The training set is the text or collection of texts. 
3. How to determine the context words of a fixed target is word is already explained. Note that a target word may appear multiple times.   
4. If *w* is the target word and *c* is a context word the basic skip-gram model uses the formula
$$ p(c|w) = \frac{\exp{({\mathbf v}_c\cdot {\mathbf v}_w)}} {\sum_{w' \in V}\exp{({\mathbf v}_{w'}\cdot {\mathbf v}_w)}} $$  
Here, ${\mathbf v}_{w'}$ is the vector representing the word $w'$ and similarly for other words. So, the fake classification task is actually determine these vectors from the weights assigned to the network when we solve the classification problem.    
5. There are two main practical issues we have to deal with. The first is that the denominator in the above equation requires that we sum over all words in teh vocabulary, which is usually quite large. The second problem is that common words like 'a', "the" etc are in the "window" of most words. The most popular solutions are negative sampling and subsampling respectively. 

In [226]:
## First we need to compute individual word or unigram probabilities.
## For this, we need word freqencies
from collections import defaultdict
#unigm_freq = defaultdict(int)
##word list in order 

word_ls = text.split()
def unigm_weight(ls):
    f_dict = defaultdict(int)
    for token in ls:
        if token in f_dict:
            f_dict[token] = f_dict[token]+  1
        else: 
            f_dict[token] = 1 
    return f_dict
# unigm_freq0["the"], len(word_ls)

In [227]:
unigm_freq0 = unigm_weight(word_ls)

In [228]:
unigm_freq0['the'], len(word_ls)

(185, 2689)

Answer= (185, 2689)

In [229]:
#function to add the frequencies raised to the power .75
def total_weight(f_dict):
    freq_raised = defaultdict(int)
    for word in f_dict:
        freq_raised[word] =  f_dict[word]**3/4
    wght =  np.sum(list(freq_raised.values()))
    return  freq_raised, wght

In [230]:
freq_raised, wght = total_weight(unigm_freq0)

In [231]:
wght

2118424.75

**Explanation.**
We want to calculate word or unigram probabilities. For a word *w* in vocabulary V, naively we will use 
$$ P(w) = \frac{freq(w)}{\sum_{w'\in V} freq(w')} $$
However, empirically it is found that a better estimate is given by the formula, 
$$ P(w) = \frac{freq(w)^{3/4}}{\sum_{w'\in V} freq(w')^{3/4}} $$
So, in the function we fill up the frequencies in a dictionary and compute the denominator, call it *wt*. We can now caluculate $P(w)$ for any word $w$ "on-the-fly" by looking up the word in the unigram frequency.  

##### <center>Subsampling </center>
Subsampling is done to get rid of some of the instances of frequently occuring words. For each word *w* in the vocabulary *V* let $z(w) = \frac{freq(w)}{|V|} $, denote the relative frequency of the word. Then define, 
$$q(w) = \min\Big(1, \big[\sqrt{\frac{z(w)}{t}} + 1 \big]\frac{t}{z(w)}\Big) $$
Here, $q(w)$ is the probability that the word *w* is kept and $t$ is threshold parameter. Take $t=.01$.  

In [232]:
#probability for subsampling, returns the function q for a numerical input
N = len(word_ls)
def prob_subsample(z):
    t = 0.005
    
    y =  np.minimum(1,(np.sqrt((z/N)/t) + 1)*(t/(z/N)))
    return y
prob_subsample(185)

0.3422599401936102

Answer = .342 (approx)

In [233]:
#Function for subsampling. Use Bernoulli sampling to keep/remove words. 
from scipy.stats import bernoulli
word_list2 = word_ls.copy()
def subsample(lst):
    unigm_freq1 = unigm_weight(lst)
    lst_count = len(lst)
    sub_ls = []
    for word in word_list2:
        prob_sub_sample = prob_subsample(unigm_freq1[word])
        keep_value = bernoulli.rvs(prob_sub_sample, size = 1)
        if keep_value == 1:
            sub_ls.append(word)
    unigm_freq1 = unigm_weight(sub_ls)
    return sub_ls, unigm_freq1
#You should also update the dictionary, unigm_freq, with new counts. 

In [234]:
word_list_sub, unigm_freq1 = subsample(word_ls)


In [235]:
len(unigm_freq0.keys()), len(unigm_freq1.keys())

(974, 974)

Answer = (974, 974)

In [236]:
len(word_list_sub), unigm_freq1["the"],unigm_freq0["the"], unigm_freq1["a"], unigm_freq0["a"]

(2420, 60, 185, 36, 62)

Answer = (2460, 77, 23), may change

In [237]:
vocab = list(unigm_freq1.keys())
len(unigm_freq0.keys()), len(unigm_freq1.keys())

(974, 974)

In [238]:
ls = ['a', 'the', 'what', 'ho', 'my', 'go', 'brown', 'cow', 'a', 'the', 'deep', 'hi', 'laugh', 'the', 'to']

In [239]:
#word encoding
def encode_word(voc):
    w_ind = {voc[i]:i for i in range(len(voc))}
    ind_w = {w_ind[x]:x for x in w_ind}

    return w_ind, ind_w

indx_word = {0: 'laugh',
 1: 'to',
 2: 'cow',
 3: 'go',
 4: 'my',
 5: 'hi',
 6: 'brown',
 7: 'ho',
 8: 'the',
 9: 'what',
 10: 'a',
 11: 'deep'}, may be different

word_indx = {'laugh': 0,
 'to': 1,
 'cow': 2,
 'go': 3,
 'my': 4,
 'hi': 5,
 'brown': 6,
 'ho': 7,
 'the': 8,
 'what': 9,
 'a': 10,
 'deep': 11}

In [240]:
#context window size = k
def context(wls, k):
    set_word = list(set(wls))
    word_indx, indx_word = encode_word(set_word)
    wls_len = len(wls)
    c_dict = defaultdict(set)
    for word in set_word:
        word_idx = word_indx[word]
        for i in range(len(wls)):
            if word == wls[i]: 
                
                context_word = []
                #create list of neighbors index

                neighbors = np.arange(i-k ,i+k+1)
                neighbors = [n for n in neighbors if n >= 0]
                neighbors = [n for n in neighbors if n < wls_len]
                neighbors = [n for n in neighbors if n != i]
                
                for n in neighbors:
                    context_word.append(wls[n])

                context_indx = []
                for context_w in context_word:
                    context_word_indx = word_indx[context_w]
                    context_indx.append(context_word_indx)


                if word_idx in c_dict:
                    c_dict[word_idx] = c_dict[word_idx]+ context_indx
                else:
                    c_dict[word_idx] = context_indx

    for i in c_dict:
        c_dict[i] = list(set(c_dict[i]))


    return c_dict, word_indx, indx_word
        

In [241]:
# di = context(ls, 2)
# di

In [242]:
context_dict,word_indx, indx_word  = context(word_list_sub, 2)

In [243]:
freq_raised1, wght1 = total_weight(unigm_freq1)
wordProb= defaultdict(int)
for word in freq_raised1:
    wordProb[word] = freq_raised1[word]/ wght1

In [262]:
#should returm m negative samples
def neg_sample(w, m):

    # create a large array A with size 100000
    A = np.zeros((100000, 1))
    instances_inA = defaultdict(int)
    # create a dictionary: the number of occurence of each word in A
    for word in wordProb:
        instances_inA[word] =  int(wordProb[word] * len(A))
        
    # fill words into A with respect to the number of occurence of each word
    begin_indx = 0
    for word in instances_inA:

        occurence = instances_inA[word]
        word_idx = word_indx[word]
        end_indx = begin_indx + occurence
        A[begin_indx:end_indx] = word_idx
        begin_indx = end_indx + 1
    
    
    #create negative samples list
    negative_sample = []
    # word index of target word:
    i = word_indx[w]
    # for each context word of word i, choose m negative samples
    for context in context_dict[i]:
        #shuffle A
        np.random.shuffle(A)
        # randomly choose samples != index of context word in A
        index_selected = np.random.choice(np.where(A!= context)[0], size = m)
        neg_words = list(A[index_selected].ravel())
        negative_sample = negative_sample + neg_words 
        
    
    return negative_sample
    

In [269]:
# For examples, get the negative samples of word gains, select 4 negative words for each context word:
neg_sample('gains', 4)

[2.0, 654.0, 373.0, 2.0, 300.0, 2.0, 654.0, 2.0, 859.0, 654.0, 2.0, 937.0]

In [246]:
# To check if the function above work well, check the index of word "gains" 
#then check the number of its context words:
gain_indx = word_indx['gains']
context_dict[gain_indx ]

[650, 783, 887]

The word "gains" has 3 context words and funtion neg_samples( ) generate 12 negative samples. Thus, funtion neg_samples( ) works well  

defaultdict(set,
            {10: {2, 6, 8, 9, 11},
             8: {0, 1, 2, 5, 7, 9, 10, 11},
             9: {4, 7, 8, 10},
             7: {3, 4, 8, 9},
             4: {3, 6, 7, 9},
             3: {2, 4, 6, 7},
             6: {2, 3, 4, 10},
             2: {3, 6, 8, 10},
             11: {0, 5, 8, 10},
             5: {0, 8, 11},
             0: {1, 5, 8, 11},
             1: {0, 8}})